In [1]:
# a Mini-RAG (Retrieval-Augmented Generation) with LlamaIndex and OpenAI

In [2]:
!pip install llama-index-llms-openai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
import importlib.metadata
print(importlib.metadata.version("llama-index"))

0.12.35


In [6]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Settings,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import CompactAndRefine

In [7]:
# Set global configuration using Settings
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=100)


In [8]:
# Load files from the "data" folder
documents = SimpleDirectoryReader("data").load_data()

# Automatically uses the splitter defined in Settings
# Splits into "nodes" (chunks of documents)
# Check how many documents were loaded
print(f"\n📄 Number of documents loaded: {len(documents)}")

# Preview the first 500 characters of the first document
if documents:
    print("\n🔍 Document Preview:\n")
    print(documents[70].text[:200])
else:
    print("⚠️ No documents loaded. Check file format and content.")




📄 Number of documents loaded: 449

🔍 Document Preview:

BLOUKRANS MASSACRE (1838). On 6 February 1838, King Di-
ngane kaSenzangakhona ordered the execution at his uMgungu-
ndlovu iKhanda of Pieter Retief and his party of Voortrekkers, who 
had been negotia


In [9]:
index = VectorStoreIndex.from_documents(documents)


In [10]:
# Use top-3 most similar chunks
retriever = VectorIndexRetriever(index=index, similarity_top_k=3)

# Synthesizer to generate citation-style answers
synthesizer = CompactAndRefine()

# Put everything together
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
)


In [11]:
response = query_engine.query("how many wars were faught by King Shaka")
print(response)

King Shaka fought two wars during his reign.


In [12]:
for i, node in enumerate(response.source_nodes):
    print(f"\n🔖 Source {i+1}:\n{node.node.get_content()[:200]}")



🔖 Source 1:
the 3rd Zulu Civil War, his Mdlalose rallied to the uSuthu cause. 
His brother Ntuzwa kaNhlaka was in command at oNdini, where 
Sekethwayo was caught up in the rout and killed.
SEKHUKHUNE woaSEKWATI. 

🔖 Source 2:
xlvi  INTRODUCTION
the south, where the British were growing apprehensive of Zulu armies 
operating near their borders. He also prized them as mercenaries with 
battle-winning firearms, and with thei

🔖 Source 3:
xxiii
Chronology
THE ZULU KINGDOM AND THE PORT NATAL SETTLERS
1816  Shaka succeeds his father, inKosi Senzangakhona kaJama, to 
the Zulu chieftainship and begins to consolidate the Zulu kingdom 
throu
